In [1]:
import pandas as pd
import numpy as np
import feather
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_pickle('pvlib.pkl')

In [3]:
df.head()

Radiation                                GH       GT    ClearSky    Ineichen  \
Datetime                  Location                                             
2010-03-19 14:15:00-10:00 AP1       476.328      NaN  973.629240  840.960548   
                          AP3       382.777      NaN  973.625962  841.600950   
                          AP4       351.610      NaN  973.624679  841.594000   
                          AP5       390.092      NaN  973.630994  841.628209   
                          AP6       353.928  343.313  973.628714  841.615852   

Radiation                             Haurwitz       Solis  
Datetime                  Location                          
2010-03-19 14:15:00-10:00 AP1       871.749955  907.496424  
                          AP3       871.733426  907.477590  
                          AP4       871.726958  907.470220  
                          AP5       871.758798  907.506499  
                          AP6       871.747297  907.493395

In [4]:
df.shape

(9058229, 6)

We need to:
  * set negative GH to 0
  * check if ClearSky is 0 when GH is 0
  * normalize GH to [0, 1] by dividing by ClearSky
  * check that GH norm is indeed in that interval

Replace negative values with 0

In [5]:
df['GH'] = df['GH'].where(df.GH >= 0, other=0)

In [12]:
df.describe().style

Radiation,GH,GT,ClearSky,Ineichen,Haurwitz,Solis
count,9.05823e+06,1.06567e+06,9.05823e+06,9.05823e+06,9.05823e+06,9.05823e+06
mean,369.962,312.06,612.044,432.497,479.78,482.42
std,351.038,344.807,372.782,348.093,365.39,374.732
min,0,-0.40968,0,0,0,0
25%,42.3027,31.5347,240.128,34.2275,76.3822,72.1068
50%,269.047,170.254,802.767,442.5,509.373,498.184
75%,619.537,519.977,900.305,762.037,816.827,832.076
max,1700.35,1587.02,1022.68,976.033,1035.09,1061.95


Filter datetimes between 7:30am and 17:30pm to always get sunlight

In [7]:
df1 = df.unstack().between_time('7:30', '17:30').stack()

Select rows where ClearSky is 0 and GH is not 0

In [11]:
res = []
for clearsky in ('ClearSky', 'Ineichen', 'Haurwitz', 'Solis'):   
    name = 'GHI_{}'.format(clearsky)
    cs = df1.loc[~np.isclose(df1['GH'], 0) &  np.isclose(df1[clearsky], 0), ['GH', clearsky]]
    
    df1[name] = np.where(np.isclose(df1[clearsky], 0), 1, df1['GH']/df1[clearsky])
    
    summ = df1[name].describe()
    q90, q99 = df1[name].quantile(q=[0.9, 0.99])
    per_gt_1, num_gt_1 = df1[name].gt(1).agg([np.mean, np.sum])
    
    summ['90%'] = q90
    summ['99%'] = q99
    summ['#>1'] = num_gt_1
    summ['%>1'] = per_gt_1
    summ['#CS=0'] = cs.shape[0]
    summ.name = clearsky
    res.append(summ)

pd.concat(res, axis=1, sort=False).style

,ClearSky,Ineichen,Haurwitz,Solis
count,6.04158e+06,6.04158e+06,6.04158e+06,6.04158e+06
mean,0.609836,0.872554,0.771898,0.772333
std,0.339151,0.403347,0.34579,0.345167
min,0,0,0,0
25%,0.318779,0.478783,0.425978,0.426563
50%,0.561595,1.05537,0.933943,0.95859
75%,0.868854,1.17201,1.04484,1.03738
max,3.16413,29.1443,12.4256,12.4105
90%,1.10763,1.28658,1.14367,1.1355
99%,1.36761,1.63198,1.34406,1.34148
